# RAG с текстовым поиском на основе Yandex Search API

В данном примере рассмотрим возможность построения RAG с использованием текстового поиска на основе [Yandex Search API](https://yandex.cloud/ru/services/search-api)  

Для данного примера необходимо:
- **GigaChat API**: получить [авторизационные данные](https://developers.sber.ru/docs/ru/gigachain/get-started/quickstart#nastroyka-avtorizatsii-zaprosov-k-giga-chat-api)
- **Yandex Search API**: получить идентификатор каталога (folderid) и API-ключ (apikey) для [аутентификации в сервисе](https://yandex.cloud/ru/docs/search-api/operations/auth)

## Конфигурация

In [1]:
from operator import itemgetter
from textwrap import dedent

from IPython.display import Markdown
from langchain_community.chat_models.gigachat import GigaChat
from langchain_community.retrievers.yandex_search_api import YandexSearchAPIRetriever
from langchain_community.utilities.yandex_search import YandexSearchAPIClient
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableParallel

Определим дополнительные функции для форматирования документов до отправки их в LLM (`format_docs`) и для формирования финального ответа (`generate_final_answer`).

In [2]:
def format_docs(docs):
    return '\n\n'.join([doc.page_content for doc in docs])

def generate_final_answer(response):
    final_answer_template = dedent(
        """
        ### Вопрос
        {question}
        
        ### Ответ
        {answer}
        
        ### Источники
        {sources}"""
    ).strip()
    
    final_answer = final_answer_template.format(
        question=response['question'],
        answer=response['answer'],
        sources='\n\n'.join(f'{doc.page_content} ({doc.metadata["url"]})' for doc in response['context'])
    )

    return final_answer

Используем модель `GigaChat-Pro` для данной задачи.

In [3]:
model = GigaChat(
    model='GigaChat-Pro',
    credentials=<авторизационные_данные>,
    verify_ssl_certs=False,
)

Создадим клиента для Yandex Search API с параметрами по умолчанию.  
Детальную информацию по дополнительным параметрам можно уточнить в [документации](https://yandex.cloud/ru/docs/search-api/concepts/get-request).

In [4]:
client = YandexSearchAPIClient(
    folderid=<folderid>,
    apikey=<apikey>,
)

Настроим retriever на основе созданного клиента, получая первые k наиболее релевантных ответов на запрос.

In [5]:
retriever = YandexSearchAPIRetriever(
    client=client,
    k=30
)

Создадим chain для генерации ответа от LLM с возможностью получения информации по документам, которые использовались при формировании ответа.

In [6]:
QA_TEMPLATE = """Ответь на следующий вопрос на основе предоставленного контекста:

{context}

Вопрос: {question}
Ответ: 
"""
prompt = ChatPromptTemplate.from_template(QA_TEMPLATE)

output_parser = StrOutputParser()

chain_without_source = (
    RunnableParallel(
        {
            "context": itemgetter("context") | RunnableLambda(format_docs),
            "question": itemgetter("question"),
        }
    )
    | prompt
    | model
    | output_parser
)
chain_with_source = (
    RunnableParallel(
        {
            "context": itemgetter("question") | retriever,
            "question": itemgetter("question"),
        }
    ).assign(answer=chain_without_source)
)

## Примеры использования

In [7]:
query = "Какие концерты проходят в Москве в июне 2024 года?"
response = chain_with_source.invoke({"question": query})
final_answer = generate_final_answer(response)
Markdown(final_answer)

### Вопрос
Какие концерты проходят в Москве в июне 2024 года?

### Ответ
В июне 2024 года в Москве проходит множество концертов и музыкальных мероприятий. Среди них - один из самых масштабных джазовых фестивалей мира "Moscow Jazz Festival", который состоится с 10 по 16 июня. Также в этот период можно посетить концерт группы "Сурганова и Оркестр" под открытым небом, где будут исполнены душевные песни разных лет. Кроме того, в июне 2024 года в Москве запланированы различные классические концерты, включая концерт-променад из цикла "Musica Antica". Полное расписание и информацию о билетах можно найти на сайтах билетных операторов и афишных ресурсах.

### Источники
Купить билеты на концерт в Москве вы круглогодично можете в кассах Ticketland и круглосуточно – на нашем сайте. (https://www.ticketland.ru/concert/2024/6/)

Поп-музыка в июне 2024 года. (https://www.ticketland.ru/concert/pop/2024/06/)

Эстрада в июне 2024 года. Найдено 13 мероприятий. (https://www.ticketland.ru/concert/estrada/2024/06/)

Купить билеты на концерт в Москве на июнь 2024, афиша концертов и цены на них. (https://www.afisha.ru/msk/schedule_concert/na-iyun/)

Подборка главных концертов и летних музыкальных фестивалей в Москве, на которые стоит сходить в июне-2024: даты, цены, обзор мероприятий. (https://daily.afisha.ru/music/27458-23-glavnyh-koncerta-moskvy-v-iyune-basta-stigmata-atl-lolita-i-ne-tolko/)

Рассказываем о самых интересных мероприятиях, которые стоит посетить в Москве в июне 2024 года. (https://daily.afisha.ru/cities/27431-chem-zanyatsya-v-moskve-v-iyune-soda-luv-motherland-beat-film-festival-i-esche-23-sobytiya/)

Кудамоскоу в курсе 13 музыкальных событий и концертов, которые пройдут в Москве в июне 2024 года. (https://kudamoscow.ru/event/concert/06-2024/)

С 10 по 16 июня 2024 года в Москве пройдёт один из самых масштабных джазовых фестивалей мира — Moscow Jazz Festival.
Кудамоскоу в курсе 77 музыкальных событий и концертов, которые пройдут в Москве 2024. (https://kudamoscow.ru/event/concert/2024/)

Кудамоскоу в курсе 8 музыкальных событий и концертов, которые пройдут в Москве 13 июня 2024 года. (https://kudamoscow.ru/event/concert/13-06-2024/)

Самые интересные концерты в Москве. Классическая музыка, вечеринки, концерты под открытым небом и в залах - мир концертов Москвы необыкновенно разнообразен.
17 лучших мультфильмов 2024 года Обзоры 17 лучших мультфильмов 2024 года. (https://www.kp.ru/afisha/msk/konczerty/iyun/)

Концерт Дельфина Концерт Дельфина 9 июня 2024, 20:00 Театр Стаса Намина.
Здесь собраны лучшие концерты Москвы по рейтингу и нашим рекомендациям. (https://www.kp.ru/afisha/msk/konczerty/luchshie-kontserty-moskvy/iyun/)

Лучшие концерты Москвы в 2024-2025 году. У нас вы можете купить билеты онлайн на концерты в Москве. (https://www.kp.ru/afisha/msk/konczerty/)

Билеты на лучшие концерты в Москве в Июне 2024 года. Успейте купить заранее по низким ценам! (https://concert.moscow/june-2024/)

ᐉ Билеты на ближайшие концерты в Москве на 2024 год. ᐉ Афиша музыкальных мероприятий столицы на Concert Moscow. ᐉ Актуальное расписание и выгодные цены на билеты! (https://concert.moscow/)

Билеты на лучшие концерты в Москве в Июле 2024 года. Успейте купить заранее по низким ценам! (https://concert.moscow/july-2024/)

Купить билеты на Концерты в Москве 13–30 июня 2024 на Яндекс Афише. (https://afisha.yandex.ru/moscow/concert/jun)

Концерты в Москве: покупка билетов на концерты онлайн, полное расписание, список концертных площадок, афиша на 2024 год на Яндекс Афише. (https://afisha.yandex.ru/moscow/concert?date=2024-06-08&period=9&filter=pop)

Концерты в Москве: покупка билетов на концерты онлайн, полное расписание, список концертных площадок, афиша на 2024 год на Яндекс Афише. (https://afisha.yandex.ru/moscow/main?rubric=concert)

Третий год подряд концерты Фестиваля будут проходить на самых популярных городских площадках Москвы.
10 июня на открытии Moscow Jazz Festival 2024 в Концертном зале им. П.И.Чайковского будет представлена совместная программа Игоря Бутмана, Московского джазового оркестра и Аллана Харриса. (https://vk.com/moscowjazzfestival)

Афиша концертов в Москве пополняется событиями, и многие артисты спешат порадовать поклонников свежим материалом. (https://kudago.com/msk/list/best-concertss-month/)

Душевные песни разных лет от нетленок до новинок — в программе концерта «Сургановой и Оркестра» под открытым московским небом.  21 июня 20:00. (https://kudago.com/msk/list/summer-list-music/)

Билеты на концерты в Москве 2024, полное расписание выступлений артистов на всех площадках города. (https://koncert-2024.ru/moscow/month-june)

Афиша концертов в Москве на июнь 2024 года, расписание всех концертов в июне в Москве. (https://concert-russia.ru/moscow/month-june)

Рассказываем о мероприятиях, которые пройдут на ней в самое ближайшее время (их действительно стоит посетить). МТС Live Лето: какие мероприятия пройдут на самой большой летней концертной площадке Москвы в 2024 году. (https://www.marieclaire.ru/stil-zjizny/mts-live-leto-kakie-meropriyatiya-proidut-na-samoi-bolshoi-letnei-koncertnoi-ploshadke-moskvy-v-2024-godu/)

Главным событием июня в Москве, бесспорно, станет открытие нового здания Третьяковской галереи на Кадашевской набережной. (https://top15moscow.ru/novosti/ozhidaemye-sobytiya-iyunya-v-moskve-2024)

Здесь вы найдете полную афишу классических концертов в Москве на июнь 2024 года: большой репертуар, онлайн продажа билетов, удобный поиск и подробное описание концертов. (https://www.belcantofund.com/calendar/2024/06/)

Концерт «Музыка лета» предлагает провести дневной досуг в обществе ведущих солистов Москонцерта, которые подготовили... (https://mosconcert.moscow/afisha/?month=6&year=2024)

В Москве интересно в любое время года.
15 июня премьера концерта-променада из цикла Musica Antica «Месье Фейе и Старинный Трактат».
С 10 по 16 июня в Москве пройдет один из самых масштабных джазовых фестивалей мира Moscow Jazz Festival 2024. (https://aif.ru/moscow/otdohnem-kulturno-afisha-samyh-yarkih-sobytiy-leta-2024-v-moskve)

Афиша концертов в Москве на июнь 2024 года, полное расписание выступлений в июне в Москве, купите билеты онлайн на официальном сайте. (https://afisha-concert.ru/moscow/list/june)

Афиша концертов в Москве июнь 2024. Низкие цены на билеты от официальных продавцов. (https://rolld.ru/msk/concert/june)

In [8]:
query = "Как прошел финал Лиги чемпионов в 2024 году?"
response = chain_with_source.invoke({"question": query})
final_answer = generate_final_answer(response)
Markdown(final_answer)

### Вопрос
Как прошел финал Лиги чемпионов в 2024 году?

### Ответ
Финал Лиги чемпионов в 2024 году прошел на стадионе "Уэмбли" в Лондоне. Мадридский "Реал" обыграл дортмундскую "Боруссию" со счетом 2:0 и стал победителем Лиги чемпионов.

### Источники
Завершился финал Лиги чемпионов УЕФА, в котором встречались дортмундская «Боруссия» и мадридский «Реал». (https://www.championat.com/football/news-5574730-borussiya-real-0-2-rezultat-finala-ligi-chempionov-sezona-2034-2034-1-iyunya-2024.html)

Так и с финалом Лиги чемпионов.
Для неё победа в ЛЧ – космический результат, тогда как для «Реала» – едва ли не стандартный. (https://www.championat.com/football/article-5574810-final-ligi-chempionov-2023-2024-real-borussiya-skolko-raz-real-i-anchelotti-vyigrali-turnir-1-iyunya-2024.html)

За финалом Лиги чемпионов — 2023/2024 предлагаем следить в онлайн-материале «Чемпионата». (https://www.championat.com/football/article-5572692-borussiya-real-madrid-1-iyunya-2024-goda-pryamaya-onlajn-translyaciya-finala-ligi-chempionov-gde-smotret-video-vremya.html)

Мадридский «Реал» стал победителем Лиги чемпионов УЕФА, обыграв в финале дортмундскую «Боруссию» со счетом 2:0. Матч проходил на стадионе «Уэмбли» в Лондоне. (https://meduza.io/news/2024/06/02/real-vyigral-ligu-chempionov)

Финал Лиги чемпионов УЕФА 2024 года прошёл 1 июня 2024 года на стадионе «Уэмбли» в Лондоне. Этот футбольный матч стал 69-м финалом Лиги чемпионов УЕФА... (https://ru.wikipedia.org/wiki/%D0%A4%D0%B8%D0%BD%D0%B0%D0%BB_%D0%9B%D0%B8%D0%B3%D0%B8_%D1%87%D0%B5%D0%BC%D0%BF%D0%B8%D0%BE%D0%BD%D0%BE%D0%B2_%D0%A3%D0%95%D0%A4%D0%90_2024)

The visual identity of the 2024 UEFA Champions League final was unveiled at the group stage draw in Monaco on 31 August 2023.[49]. (https://en.m.wikipedia.org/wiki/2024_UEFA_Champions_League_final)

Финал турнира прошёл в Лондоне, на стадионе «Уэмбли»[1]. Победитель квалифицируется в групповой этап Лиги чемпионов УЕФА следующего сезона, а также получит право принять участие в матче за Суперкубок УЕФА 2024 против победителя Лиги Европы УЕФА 2023/24. (https://ru.wikipedia.org/wiki/%D0%9B%D0%B8%D0%B3%D0%B0_%D1%87%D0%B5%D0%BC%D0%BF%D0%B8%D0%BE%D0%BD%D0%BE%D0%B2_%D0%A3%D0%95%D0%A4%D0%90_2023/2024)

После трансформации Кубка европейских чемпионов в Лигу чемпионов (1991) на именитом стадион прошел третий финал (2011, 2013). (https://www.rbc.ru/sport/02/06/2024/665b27549a79473af88e70a7)

«Реал» в рекордный, 15-й раз выиграл Лигу чемпионов, победив в финале дортмундскую «Боруссию». (https://www.rbc.ru/sport/02/06/2024/665bfaf89a7947522a294cb9)

«Реал» впервые сыграл финал Кубка европейских чемпионов / Лиги чемпионов на «Уэмбли», который в восьмой раз принял решающий матч главного еврокубкового турнира. (https://www.rbc.ru/sport/01/06/2024/665a9a409a7947cd4cf1e99d)

Этот финал также стал особенным для "Боруссии Дортмунд", так как это был последний матч для двух легендарных игроков клуба - Марко Ройса и Матса Хуммельса. (https://dzen.ru/a/ZlwkgtA-jxLskAJ2)

Финал Лиги чемпионов 2024 года состоялся в субботу, 1 июня, на стадионе «Уэмбли» в Лондоне. (https://dzen.ru/a/ZluU0dA-jxLsd3EH)

13 февраля 2024 года состоится матч 1/8 финала Лиги чемпионов между «Манчестер Сити» и «Копенгагеном». Для многих выход датской команды в следующую стадию стал удивлением, но, скорее всего, их сказка подходит к концу. «Горожане» подошли к матчу в отличном настроении – восемь побед подряд в Лиге чемпионов, оптимальный состав. Еще один матч 1/8 финала пройдет между «Лейпцигом» и мадридским «Реалом». (https://dzen.ru/list/trends/kto-vyigral-ligu-chempionov-v-2024)

Обзор финального матча Лиги чемпионов 01.06.2024» на канале «МАТЧ!» в хорошем качестве и бесплатно, опубликованное 2 июня 2024 года в 0:57, длительностью 00:10:19, на видеохостинге RUTUBE. (https://rutube.ru/video/6151764eaa879d532d93b3cca308628b/)

Мадридский "Реал" обыграл "Боруссию" из Дортмунда в финале Лиги чемпионов УЕФА со счетом 2:0 и в 15-й раз завоевал главный клубный трофей Европы. (https://www.dw.com/ru/madridskij-real-stal-pobeditelem-ligi-cempionov-uefa/a-69243033)

Финал Лиги чемпионов в 2025 году пройдет в Мюнхене на «Альянц Арене». С сезона-2024/25 формат Лиги чемпионов претерпит изменения — число участников увеличится до 36, они будут играть в одной группе по швейцарской системе. (https://sportmail.ru/news/football-eurocups/61334157/)

Мадридский "Реал" обыграл дортмундскую "Боруссию" в финале футбольной Лиги чемпионов.Встреча прошла в Лондоне на стадионе "Уэмбли" и завершилась победой "королевского клуба" со счетом 2:0. Голы забили Даниэль Карвахаль (74-я минута) и Винисиус Жуниор (83). (https://rsport.ria.ru/20240601/match-1949944744.html)

В 2024 году "сливочные" победили не только в ЛЧ, но и в Ла Лиге и Суперкубке Испании
Вишенка на торте: "Реал" взял "ушастый" трофей, пройдя весь розыгрыш ЛЧ без поражений. (https://rsport.ria.ru/20240602/real-1949933363.html)

Подошел к концу финал Лиги чемпионов сезона-2022/23.
РИА Новости Спорт рассказывает, как это было.Не прошло и трех лет...Финал Лиги чемпионов доехал до пролива Босфора. (https://rsport.ria.ru/20230611/final-1877447789.html)

Мадридский "Реал" обыграл дортмундскую "Боруссию" со счетом 2:0 и стал победителем Лиги чемпионов. (https://ren.tv/news/sport/1226014-real-obygral-borussiiu-i-stal-pobeditelem-ligi-chempionov)

В финале Лиги чемпионов он обыграл «Боруссию». Свой очередной трофей лучшего клуба Европы «Реал» завоевал в матче с банальной концовкой, но интересным сюжетом.
Выступление американского певца Ленни Кравица перед началом финального матча Лиги чемпионов. (https://www.kommersant.ru/doc/6743994)

"Уэмбли" примет финал 2024 года.
Лига чемпионов УЕФА сезона 2023/24 завершится финалом на легендарном стадионе "Уэмбли" 1 июня. (https://ru.uefa.com/uefachampionsleague/match/2039970--dortmund-vs-real-madrid/final/)

Лига чемпионов УЕФА - Что надо знать о финале Лиги чемпионов-2024 - Новости.
Rudimental станет хедлайнером первого за пять лет Friday Night Show, которое состоится 31 мая и откроет уик-энд, посвященный финалу Лиги чемпионов в Лондоне . (https://ru.uefa.com/uefachampionsleague/news/0282-1827072c4017-5a3b442ec8c7-1000--%D1%87%D1%82%D0%BE-%D0%BD%D0%B0%D0%B4%D0%BE/)

Официальный справочник по Лига чемпионов УЕФА-2023/24 с результатами, статистикой и видео. (https://ru.uefa.com/uefachampionsleague/history/seasons/2024/)

Мадридский «Реал» одержал победу над дортмундской «Боруссии» в финале Лиги чемпионов 2024 со счетом 2:0. Испанский клуб стал лучшим в Европе в 15-й раз. Игра финала Лиги чемпионов прошла на стадионе «Уэмбли» в Лондоне.
Он одержал шесть побед в финалах турнира, последнюю — в 1966 году, сообщает РБК. (https://vtomske.ru/news/204646-real-vyigral-final-ligi-chempionov-2024)

«Реал» одержал победу над «Боруссией» в финале Лиги чемпионов — 2:0. (https://www.sport-express.ru/football/champions-league/news/borussiya-real-rezultat-matcha-finala-ligi-chempionov-1-iyunya-2024-2217659/)

«Боруссия» — «Реал»: финал Лиги чемпионов сезона 2023 / 2024 — дата и место проведения, стадион «Уэмбли» (Лондон), время начала и онлайн трансляция матча, результат и главные материалы — новости и обзор игры 1 июня. (https://www.sport-express.ru/football/champions-league/stories/borussiya-real-final-ligi-chempionov-2024-rezultat-i-glavnye-materialy-novosti-i-obzor-matcha-1-iyunya-2208494/)

«Реал» в 15-й раз стал победителем Лиги чемпионов.
Спустя два года финал вновь преподнес зрителям дерби. (https://www.sport-express.ru/football/champions-league/reviews/real-v-15-raz-vyigral-ligu-chempionov-mnenie-o-pobede-madridcev-nad-borussiey-1-iyunya-2024-goda-2217680/)

Мадридский "Реал" стал победителем Лиги чемпионов. В финале испанский клуб со счетом 2:0 обыграл дортмундскую "Боруссию". (https://rg.ru/2024/06/02/madridskij-real-v-15-j-raz-vyigral-ligu-chempionov.html)

Мадридский "Реал" обыграл немецкую "Боруссию" со счетом 2:0 в финале Лиги чемпионов. (https://rg.ru/2024/06/02/koroli-evropy.html)

In [9]:
query = "Что такое gigachain?"
response = chain_with_source.invoke({"question": query})
final_answer = generate_final_answer(response)
Markdown(final_answer)

### Вопрос
Что такое gigachain?

### Ответ
GigaChain, также известный как GigaChain (SDK), является фреймворком для разработки приложений с использованием больших языковых моделей (LLM), таких как GigaChat или YandexGPT. Он предоставляет стандартный интерфейс для создания памяти, коллекцию реализаций памяти и примеры цепочек и агентов, которые используют память. GigaChain можно установить с помощью менеджера пакетов pip. Этот фреймворк позволяет интегрироваться со сторонними ресурсами, такими как локальные и удаленные файловые системы, API и базы данных. GigaChain находится в состоянии альфа-версии и в настоящее время адаптируется для работы с GigaChat.

### Источники
GigaChain предоставляет стандартный интерфейс для создания памяти, коллекцию реализаций памяти и примеры цепочек и агентов, которые используют память. (https://developers.sber.ru/docs/ru/gigachain/overview)

GigaChain можно установить с помощью менеджера пакетов pip (https://developers.sber.ru/docs/ru/gigachain/get-started/quickstart)

GigaChain позволяет интегрироваться со сторонними ресурсами такими как локальные и удаленные файловые системы, API и базы данных. (https://developers.sber.ru/docs/ru/gigachain/security)

GigaChain находится в состоянии альфа-версии: мы заняты переводом библиотеки и ее адаптацией для работы с GigaChat. (https://github.com/ai-forever/gigachain)

GigaChain это фреймворк для разработки приложений с использованием больших языковых моделей (LLM), таких как GigaChat или YandexGPT. (https://github.com/andrei-zimin/gigachain)

GigaChain это фреймворк для разработки приложений с использованием больших языковых моделей (LLM), таких как GigaChat или YandexGPT. (https://github.com/NewSocOps/gigachain)

SDK GigaChain отличается от LangChain тем, что он переведен на русский язык, адаптирован к экосистеме русскоязычных языковых моделей и, конечно же, к GigaChat . (https://habr.com/ru/companies/sberdevices/articles/794773/)

По словам руководителя блока «Технологии» Сбербанка Андрея Белевцева, помимо GigaChat API разработан и GigaChain (SDK) — адаптированная версия библиотеки LangChain для русского языка с поддержкой GigaChat API, которая позволит разработчикам создавать собственные ИИ‑приложения, использующие технологии «Сбера». (https://habr.com/ru/news/762536/)

привет Минцифры !gigachain install-rus-certs. (https://habr.com/ru/articles/800657/)

GigaChain — это версия на базе библиотеки LangChain для работы с русским языком, что позволяет использовать её при работе и с другими языковыми моделями. (https://courses.sberuniversity.ru/llm-gigachat/2/4/1)

А вот как выглядит написание промптов в GigaChain. (https://courses.sberuniversity.ru/llm-gigachat/2/4/2)

Итак, цепочка рассуждений (Chain of thoughts, CoT) — это метод, при котором GigaChat генерирует несколько промежуточных шагов-размышлений, прежде чем даёт финальный ответ на запрос. (https://courses.sberuniversity.ru/llm-gigachat/2/5/1)

Gigachain - это фреймворк на основе библиотеки Langchain для создания приложений с применением больших языковых моделей. (https://www.youtube.com/watch?v=eo5LyjRobic)

После каждого ролика мне приходят вопросы о том, как "скормить" разные файлы большой языковой модели. В прошлом туториале мы уже рассмотрели общую структуру ... (https://www.youtube.com/watch?v=PHCm7dADiVk)

pip install gigachain or pip install langsmith &amp;&amp; conda install gigachain -c conda-forge. What is this? Large language models (LLMs) are emerging as a transformative technology, enabling developers to build applications that they previously could not. (https://pypi.org/project/gigachain/)

This package holds experimental GigaChain code, intended for research and experimental uses. (https://pypi.org/project/gigachain-experimental/)

gigachain_community-0.2.0.tar.gz (1.4 MB просмотр хэшей). (https://pypi.org/project/gigachain-community/)

Кроме того, также представлен GigaChain (SDK) — это адаптированная версия библиотеки LangChain для русского языка с поддержкой GigaChat API. (https://kod.ru/gigachat-api-gigachain)

The common name of the sons of Giga Chad, whose number is known only to Giga Chad himself. The Gigachain includes people whose character, behavior, appearance and moral values can compete only with the Giga Chad itself. (https://www.urbandictionary.com/define.php?term=Gigachain)

Чтобы упростить и ускорить разработку таких приложений, команда GigaChat разработала SDK GigaChain. (https://edutoria.ru/lp/gigachat-api-sdk/)

Gigachain is a technology platform using blockchain to protect workers worldwide from exploitation and labour abuse. (https://twitter.com/gigachain_news)

В библиотеках GigaChain/LangChain для этого есть специальные инструменты под названием загрузчики. (https://rutube.ru/video/985ce2480fc21216ceccb7cb43172f95/)

Gigachain - это фреймворк на основе библиотеки Langchain для создания приложений с применением больших языковых моделей. (https://rutube.ru/video/071dc4279fe2dc87ef7afb7cabd98fd7/)

Библиотека GigaChain является модифицированной версией библиотеки LangChain, представляющей собой мощный инструмент для обработки данных из разных источников. (https://rutube.ru/video/2760bf2c77647568813ecc84682a82a4/)

Несколько готовых программ представлены на GitHub в разделе описания фреймворка gigachain. (https://pikabu.ru/story/gigachat_api_otkryili_dlya_fizicheskikh_lits_besplatno_10832509)

GigaChain (GIGA) is a cryptocurrency, deployed on Binance Smart Chain(BSC BEP-20). (https://thebittimes.com/token-GIGA-BSC-0x482fdac39b7965a7cb31a8601be6a877a764a5de.html)

В рамках хакатона мы создали своего ИИ-ассистента для научных исследований с помощью фреймворка GigaChain. (https://tenchat.ru/media/2252588-osvoyeniye-otechestvennykh-instrumentov-gigachat-i-gigachain-na-deephackagents)

Для облегчения использования этих моделей, Сбер предоставил доступ к адаптированной версии библиотеки LangChain под русский язык — GigaChain. (https://news.rambler.ru/tech/51954824-neyroset-sbera-stala-dostupna-dlya-razrabotchikov/)

GIGACHAIN GIGACHAIN is a crypto coin (token) that running on Ethereum ERC-20 Increase GIGACHAIN reach! (https://recentcoin.com/token/gigachain-gigachain-0xdde2c8b2)

On top of that, one unique thing about Gigachains is that it exclusively offers currency trading. (https://www.financebrokerage.com/gigachains-review/)